<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/MKVExtract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ⬅️ **Mount Google Drive**
Run the following command to mount google drive:

In [1]:
#@title **[0] Link your Google Drive**
#@markdown <center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#@title **[1] Run this code if you want a longer run time (Optional but recommended)</font>**
#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

### ⬅️ **MKV Extractor**

In [4]:
#@markdown <h3>➡️ Click Here to Install FFmpeg, VCSI & Mkvtoolnix</h3>

from IPython.display import clear_output
import os, sys, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing ffmpeg...", ty='twg')
!apt install ffmpeg &> /dev/null
textAn("Installing mkvtoolnix...", ty='twg')
!sudo apt-get install mkvtoolnix &> /dev/null
textAn("Installing vcsi...", ty='twg') 
!pip install vcsi &> /dev/null


from IPython.display import HTML, clear_output
clear_output()
print("Server Started Successfully")

Server Started Successfully


In [ ]:
import os
import re
from pathlib import Path

os.path.isdir('.logfiles')
True

#@title <h5>➡️ Generate MKV Information</h5>
#@markdown <h5 style="font-size:10px">This option is to generate single or batch files information, where you provide the InputLocation location.</h5>

#@markdown - - -

InputLocation = "/content/drive/Shareddrives/01. Anime Server/01. TV Show/35. RWBY/Season 1v2"  # @param {type:"string"}

SearchForSubfolder = False #@param {type:"boolean"}
#@markdown &ensp;&ensp;Check search for subfolder will make it search the files inside the <b>subdirectory</b> where you provide <b>path location</b>.

#@markdown <br>
#@markdown <h5 style="font-size:10px">Which option you want to generate information.</h5>

#@markdown - - -
Audio = False  # @param {type:"boolean"}

Subtitle = False  # @param {type:"boolean"}

Attachment = False  # @param {type:"boolean"}

if SearchForSubfolder:
    ext = '**/*.mkv'
else:
    ext = '*.mkv'


def detect_available_audio_tracks(source_video):

    if source_video.endswith('.mkv'):
        file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
        AvailableAudioTracks = re.findall(
            r"Track ID ([0-9]+): audio \((.+)\)", file, re.MULTILINE)
        Filename = re.findall(r"File '(.+)'", file)
        audio_tracks = ''
        for track in AvailableAudioTracks:
            audio_tracks += f'\033[1;0mAudio track {track[0]} \033[1;31mformat \033[1;32mtype \033[1;34m{track[1]}\n'

        if audio_tracks:

            print(
                    f'\n\033[1;33mThese audios were found in the file {Filename[0]} \n{audio_tracks}')

    elif os.path.isdir(source_video):
        for source_video in Path(source_video).glob(ext):
            file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
            AvailableAudioTracks = re.findall(
                r"Track ID ([0-9]+): audio \((.+)\)", file, re.MULTILINE)
            Filename = re.findall(r"File '(.+)'", file)
            audio_tracks = ''
            for track in AvailableAudioTracks:
                audio_tracks += f'\033[1;0mAudio track {track[0]} \033[1;31mformat \033[1;32mtype \033[1;34m{track[1]}\n'

            if audio_tracks:

                print(
                        f'\n\033[1;33mThese audios were found in the file {Filename[0]} \n{audio_tracks}')

    else:
        print('error')


def detect_available_subtitle_tracks(source_video):

    if source_video.endswith('.mkv'):
        !ffprobe '{source_video}' 2>&1 >/dev/null | grep Stream | grep Subtitle > ".logfiles/subtitles.txt"
        subtitles = '.logfiles/subtitles.txt'
        print("\033[1;33mThese subtitles are found in the file ", source_video)
        with open(subtitles,"r") as logs:
          count = 0
          for l in logs:
            count += 1
            res = re.search('(\(.+?\)):', l).group(1)
            l = l.strip()
            src =f"{l}"
            out = src.replace("Stream #0:", "\033[1;0mSubtitle track ")
            out = out.replace(res+':', "")
            out = out.replace('Subtitle:', "\033[1;32m language" + "\033[1;34m" + res + "\033[1;0m format type\033[1;31m ")
            print(out)
          print("\n")

    elif os.path.isdir(source_video):
        for source_video in Path(source_video).glob(ext):
            !ffprobe '{source_video}' 2>&1 >/dev/null | grep Stream | grep Subtitle > ".logfiles/subtitles.txt"
            subtitles = '.logfiles/subtitles.txt'
            print("\033[1;33mThese subtitles are found in the file ", source_video)
            with open(subtitles,"r") as logs:
              count = 0
              for l in logs:
                count += 1
                res = re.search('(\(.+?\)):', l).group(1)
                l = l.strip()
                src =f"{l}"
                out = src.replace("Stream #0:", "\033[1;0mSubtitle track ")
                out = out.replace(res+':', "")
                out = out.replace('Subtitle:', "\033[1;32m language" + "\033[1;34m" + res + "\033[1;0m format type\033[1;31m ")
                print(out)
              print("\n")

    else:
        print('error')


def detect_available_attachment_tracks(source_video):

    if source_video.endswith('.mkv'):
        file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
        AvailableAttachment = re.findall(
            r"Attachment ID ([0-9]+):.+ file name '(.+)'", file, re.MULTILINE)
        Filename = re.findall(r"File '(.+)'", file)
        Attachment = ''
        for att in AvailableAttachment:
            Attachment += f'\033[1;0mAttachment track {att[0]} \033[1;32mname \033[1;34m{att[1]}\n'

        if Attachment:

            print(
                    f'\n\033[1;33mThese attachments were found in the file {Filename[0]} \n{Attachment}')

    elif os.path.isdir(source_video):
        for source_video in Path(source_video).glob(ext):
            file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
            AvailableAttachment = re.findall(
                r"Attachment ID ([0-9]+):.+ file name '(.+)'", file, re.MULTILINE)
            Filename = re.findall(r"File '(.+)'", file)
            Attachment = ''
            for att in AvailableAttachment:
                Attachment += f'\033[1;0mAttachment track {att[0]} \033[1;32mname \033[1;34m{att[1]}\n'

            if Attachment:

                print(
                        f'\n\033[1;33mThese attachments were found in the file {Filename[0]} \n{Attachment}')

    else:
        print('error')


if Audio:
    detect_available_audio_tracks(InputLocation)

if Subtitle:
    detect_available_subtitle_tracks(InputLocation)

if Attachment:
    detect_available_attachment_tracks(InputLocation)


##### ⬅️ **MKVToolnix Mode**

In [ ]:
import os, shlex, json, sys
import subprocess as sp

#@title <h5>➡️ Start MKV Subtitle Extract</h5>
#@markdown <h5 style="font-size:10px">This option is to extract batch file, where you provide the path location.</h5>

#@markdown - - -
#@markdown &ensp;&ensp;This is path to <b>directory</b> where your file exists.
path = "/content/drive/Shareddrives/01. Anime Server/04. Anime China/War God System!" #@param {type:"string"}
#@markdown &ensp;&ensp;This is path to directory where you will <b>save</b> the extracted files.
out_path = "/content" #@param {type:"string"}
#@markdown &ensp;&ensp;It will search the files inside the <b>subdirectory</b> where you provide <b>path</b>.
sub_folders = False #@param {type:"boolean"}
#@markdown &ensp;&ensp;Choose to extract <b>default</b> subtitle or all subtitles that exists.
extract_all_sub = False #@param {type:"boolean"}

verbose = False #@param {type:"boolean"}

keep_folders = False #@param {type:"boolean"}

def scan_for_files():
    # Scan for mkv files in the user given in path
    scanned_files = 0

    if sub_folders == True:
        for root, _, files in os.walk(path):
            for f in files:
                if f.endswith('.mkv'):
                    print('\n' + 'Found file: "' + f + '" \nin "' + root + '"')
                    file_info = get_mkv_info((root + '/'), f)
                    scanned_files += 1
                    create_command(f, file_info, root)
    else:
        for f in os.listdir(path):
            if f.endswith('.mkv'):
                print('\n' + 'Found file: "' + f + '" \nin "' + path + '"')
                file_info = get_mkv_info((path + '/'), f)
                scanned_files += 1
                create_command(f, file_info, path)


    if scanned_files == 0:
        print('\n' + 'No compatible files found')
    else:
        print('\n' + str(scanned_files) + ' file(s) processed')


def get_mkv_info(f_path, file):
    file_path = os.path.join(f_path, file)
    cmd = 'mkvmerge -i -F json "' + file_path + '"'
    new_process = sp.Popen(shlex.split(cmd), stdout=sp.PIPE)

    try:
        json_out = json.loads(new_process.communicate(timeout=30)[0].decode())
        if new_process.returncode != 0:
            new_process.kill()

        file_info = process_stdout(json_out)
        return file_info

    # TODO: Handle errors I don't know about because I can't get any to show up
    except TimeoutError as error:
        new_process.kill()
        sys.exit(str(error))
    except sp.TimeoutExpired as error:
        new_process.kill()
        sys.exit(str(error))


def process_stdout(json_out):
    print('Processing track info')
    track_dict = {}
    file_info = []

    for t in json_out['tracks']:
        i = str(t['id'])
        track_dict[i] = {'type': t['type'],
                         'codec_id': t['properties']['codec_id'],
                         'default_track': t['properties']['default_track'],
                         'language': t['properties']['language']}
        try:
            track_dict[i]['track_name'] = t['properties']['track_name']
        except KeyError:
            track_dict[i]['track_name'] = ''


    file_info.append(track_dict)

    try:
        mkv_title = json_out['container']['properties']['title']
    except KeyError:
        mkv_title = ''
    file_info.append(mkv_title)

    return file_info



def create_command(file, file_info, root):
    track_dict, mkv_title = file_info

    mkvextract_cmd = ['mkvextract tracks "' + os.path.join(root, file) + '" ']


    for track in track_dict:
        name = track_dict[track]['track_name']
        lang = track_dict[track]['language']
        is_def = 'no'
        if (track_dict[track]['default_track']) is True:
            is_def = 'yes'

        # Can't remember why I decided to use a while loop here but it works so I don't care
        while True:
            if track_dict[track]['type'] == 'subtitles':
                    if extract_all_sub == True:
                        mkvextract_cmd.append(create_sub_cmd(root, file, track, track_dict[track]))
                        break
                    else:
                        if is_def == 'yes':
                            mkvextract_cmd.append(create_sub_cmd(root, file, track, track_dict[track]))
                            break

                        else:
                          break
            else:
               break 
               
    if  len(mkvextract_cmd) > 1:
        call_program(mkvextract_cmd, '[MKVExtract] ')
    else:
        print('[MKVExtract] No matching subtitles found, skipping the call to MKVExtract')


def create_sub_cmd(root, file, track, track_info):
    codec = track_info['codec_id']
    codec_ext = {'S_TEXT/UTF8': '.srt',
                 'S_TEXT/SSA': '.ssa',
                 'S_TEXT/ASS': '.ass',
                 'S_TEXT/USF': '.usf',
                 'S_TEXT/WEBVTT': '.vtt',
                 'S_VOBSUB': '.idx',
                 'S_HDMV/PGS': '.sup'}
    try:
        ext = codec_ext[codec]
        if extract_all_sub == True:
            if keep_folders == True:
                cmd = track + ':"' + out_path + os.path.join(root, file.rsplit('.mkv')[0]) + '.' + track + '_' + track_info['language'] + ext + '"'
            else:
                cmd = track + ':"' + os.path.join(out_path, file.rsplit('.mkv')[0]) + '.' + track + '_' + track_info['language'] + ext + '"'
        else:
            if keep_folders == True:
                cmd = track + ':"' + out_path + os.path.join(root, file.rsplit('.mkv')[0]) + ext + '"'
            else:
                cmd = track + ':"' + os.path.join(out_path, file.rsplit('.mkv')[0]) + ext + '"'
        
        print('  Adding subtitle "' + track + '_' + track_info['language'] + ext + '" to MKVExtract call')
        return cmd
    except KeyError:
        print('Unknown subtitle codec (' + codec + '), or is not supported by MKVMerge')

def call_program(cmd, program):
    line = ''

    cmd = list(filter(None, cmd))
    if len(cmd) > 1:
        cmd = ' '.join(cmd)
        print(cmd)

    try:
        with sp.Popen(shlex.split(cmd), stdout=sp.PIPE, stderr=sp.PIPE, universal_newlines=True) as p:
            for line in p.stdout:
                if verbose == True:
                    if line.startswith('Progress:'):
                        if '100%' in line:
                            print('\r' + program + line.rstrip('\n'), end='')
                        else:
                            print('\r' + program + line.rstrip('\n'), end='')
                    elif line == '\n':
                        print('')
                    elif line.startswith('Error'):
                        continue
                    else:
                        print(program + line, end='')
        print('')

        if p.returncode != 0:
            p.kill()
            sys.exit(str(line).lstrip('Error:'))

    except Exception as error:  # TODO: What Exceptions to handle here?
        sys.exit(str(error))
scan_for_files()

##### ⬅️ **FFmpeg Mode**

In [ ]:
#@title <h5>➡️ Start MKV Extract</h5>

from IPython.display import clear_output

#Input and output file
FilePath = "" #@param {type:"string"}
SameFolderExtract = False #@param {type:"boolean"}
#@markdown &ensp;&ensp;<i>You can leave output path empty if extract in the same folder is checked
OutputPath = "/content" #@param {type:"string"}
from pathlib import Path
Path(OutputPath).mkdir(parents=True, exist_ok=True)
OutputType = input("Please enter export file type... \nExample (srt, ass, vtt, etc) \nFile type: ")
Type = '.' + OutputType

#Track Initiation
StartTrack = 2 #@param {type:"integer"}
StartTrack = StartTrack-1
EndTrack = 6 #@param {type:"integer"}
EndTrack = EndTrack

#Renaming file
Fix = len(".mkv")
src =f"{FilePath}"
dst =f"{src[:-(int(Fix))]}"
if SameFolderExtract == True:
  dst =f"{dst}_track"
else:
  sparator = '/'
  for i in range(1,11):
    # Remove all characters before the character '-' from string
    before, sparator, after = dst.partition('/')
    if len(after) > 0:
      dst = after
  dst =f"{OutputPath}/{dst}_track"
!ffprobe '{FilePath}' 2>&1 >/dev/null | grep Stream > namingspecslist.txt
import re
for Tracks in range(StartTrack,EndTrack):
  StartTrack = StartTrack+1
  Track = StartTrack
  x = Track+1
  fname = open("namingspecslist.txt")
  cuk = !sed -n {x}p namingspecslist.txt
  specs = str(cuk)
  pattern = '\(.*?\)'
  findid = re.search(pattern, specs)
  specsresult = findid.group()
  scr =f"{specsresult}"
  id = scr.replace("(", "")
  id = id.replace(")", "")
  !ffmpeg -y -v error -stats -txt_format text -i '{FilePath}' -map 0:{Track} '{dst}{Track}.{id}{Type}'

### ⬅️ **File Rename**

In [ ]:
#@title <h3>➡️ Renaming File</h3>
import os
import signal
import sys

def sigint_handler(signal, frame):
    print ('KeyboardInterrupt is caught')
    sys.exit(0)
signal.signal(signal.SIGINT, sigint_handler)

InputLocation = "/content" #@param {type:"string"}

Text2Replace  = "[LENDRIVE] " #@param {type:"string"}
ReplacementText   = "[-TJPLAY-] " #@param {type:"string"}

FileType = "srt" #@param ["Write it yourself...", "mp4", "mkv", "png", "jpg", "jpeg", "ass", "srt"]
if FileType == "Write it yourself...":
  ftype = input('Enter the file type to rename: ') 
  Extension = '.' + ftype
else:
  Extension = '.' + FileType

PutLabel = False #@param {type:"boolean"}
Label = "" #@param {type:"string"} 

fix = len(Extension)
#-(int(fix))

# Renaming the file

def RenameMe():
  for count, filename in enumerate(os.listdir(InputLocation)):
    if(filename.endswith(Extension)):
          src =f"{InputLocation}/{filename}"  # foldername/filename, if .py file is outside folder
          cuk =f"{filename}"
          fname = cuk.replace(Text2Replace, ReplacementText)
          dst =f"{InputLocation}/{fname}"
          
          # rename() function will
          # rename all the files with the pre defined extension
          os.rename(src, dst)


def LabelMe():
  for count, filename in enumerate(os.listdir(InputLocation)):
    if(filename.endswith(Extension)):
          src =f"{InputLocation}/{filename}"  # foldername/filename, if .py file is outside folder
          dst =f"{InputLocation}/{Label} {filename}"
          
          # rename() function will
          # rename all the files with the pre defined extension
          os.rename(src, dst)

if PutLabel == False:
  RenameMe()
else:
  LabelMe()